##SUPERMARKET SALES ANALYSIS

Importando librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import numpy

Variables Globales

In [2]:
global df_ventas
global df_ventas_limpio

*Importando la Base de Datos .csv*

In [3]:
def obtencion_datos():
    global df_ventas
    df_ventas = pd.read_csv('reporte_ventas.csv', sep = ',')
obtencion_datos()

Normalización y limpieza de los datos

In [4]:
#Formateamos el tipo de dato de las fechas a uno adecuado
df_ventas['fecha_pedido'] = pd.to_datetime(df_ventas['fecha_pedido'], format='%Y-%m-%d')
df_ventas['fecha_envio'] = pd.to_datetime(df_ventas['fecha_envio'], format='%Y-%m-%d')

#Eliminar repetidos
df_ventas_limpio = df_ventas.drop_duplicates()

#Eliminamos registros de prueba
def eliminar_registros_prueba(df):
  return df.drop(df[df['nombre_cliente'] == 'prueba'].index)
df_ventas_limpio = eliminar_registros_prueba(df_ventas_limpio)

#Reemplazamos los nombres de clientes NaN por el valor "missing"
df_ventas_limpio['nombre_cliente'].fillna('missing', inplace=True)

In [5]:
# Filtrar los registros con total_ganancia > 0
df_ventas_limpio = df_ventas.drop(df_ventas[df_ventas["total_ganancia"]  <= 0].index)

#reiniciamos el index
df_ventas_limpio.reset_index(inplace=True)
df_ventas_limpio.drop(columns=['index'], inplace=True)

In [ ]:
df_ventas_limpio

1. ¿Cuáles son las ventas totales por año? ¿Y qué año tuvo mejor desempeño? (Pablo)

In [7]:
#Code here
#df_ventas_limpio

2. ¿Cuáles son los 7 productos que más ganancias mostraron durante todo el periodo?

In [8]:
#Code here
#df_ventas_limpio

3. ¿Si separamos las ventas totales por año y por regiones ¿Qué insight podemos obtener de nuestras ventas? (Jhojan)

In [9]:
#Code here
#df_ventas_limpio

4. ¿Cuál es el método de envío más utilizado por los clientes del Supermercado? ¿Es proporcional para B2B y B2C?

In [10]:
#Code here
#df_ventas_limpio

5. ¿Cuáles son las ventas totales por trimestre en el estado de Sao Paulo? (Everardo)

##PREPARACIÓN DE LOS DATOS

In [11]:
#Función pregunta_5()
def pregunta_5():
    # Definimos las variables globales
    global df_ventas
    global df_ventas_limpio
#Creamos una copia del dataframe df_ventas_limpio:
df_ventas_sp = df_ventas_limpio.copy()
#Filtramos los registros con valor São Paulo en la columna estado:
df_ventas_sp = df_ventas_sp[df_ventas_sp['estado'] == 'São Paulo']

#Establecemos la columna fecha_pedido como el nuevo índice:
df_ventas_sp.set_index('fecha_pedido', inplace=True)
#Redondear el valor de la columna total_venta y resetear el índice
df_ventas_sp = df_ventas_sp.select_dtypes(include=['int64', 'float64']).resample('Q').agg('sum', numeric_only=True)
df_ventas_sp['total_venta'] = df_ventas_sp['total_venta'].round()

df_ventas_sp.reset_index(inplace=True)


In [ ]:
print(df_ventas_sp[['fecha_pedido', 'total_venta']])


##VISUALIZACIÓN

In [ ]:
# Función
def pregunta_5():
    global df_ventas
    global df_ventas_limpio

    # Copia dataframe df_ventas_limpio con nuevo nombre df_ventas_sp
    df_ventas_sp = df_ventas_limpio.copy()
    df_ventas_sp = df_ventas_sp.query('estado == "São Paulo"')[["fecha_pedido", "total_venta"]]
    # Establece la columna fecha_pedido como índice del DataFrame df_ventas_sp
    df_ventas_sp.set_index('fecha_pedido', inplace=True)
    # Aplica la función sum en el dataframe para calcular la venta total por trimestre, redondea a dos decimales
    df_ventas_sp = df_ventas_sp.resample('Q').agg('sum')
    df_ventas_sp['total_venta'] = round(df_ventas_sp['total_venta'] / 1e3, 2)
    # Restablece el indice
    df_ventas_sp = df_ventas_sp.reset_index()

    # Área del gráfico
    fig, ax = plt.subplots(figsize=(14, 4))

    # Gráfico de linea
    puntos_a_marcar = [True if mes.month == 12 else False for mes in pd.to_datetime(df_ventas_sp['fecha_pedido'])]
    ax.plot(df_ventas_sp['fecha_pedido'], df_ventas_sp['total_venta'], lw=2, color='#078c62', marker='o',
    markersize=10, markerfacecolor='#078c62', markevery=puntos_a_marcar)

    # customizer
    ax.set_title('Ventas por trimestre en el estado de São Paulo', fontsize=18, color='#212529', loc='left')
    ax.text(0.3, 0.95, 'De 2019 a 2022 (en miles de dólares)', transform=ax.transAxes, color='#0521ad', fontsize=14)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_frame_on(False)
    ax.xaxis.set_tick_params(labelsize=14, labelcolor='#05114a')
    ax.tick_params(axis='both', which='both', length=0)
    ax.yaxis.grid(False)
    ax.xaxis.grid(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.ylim(0, 400)

    # Definiendo el intervalo entre trimestres y colocando los nombres de los meses en los labels
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    x_labels = ['Mar', 'Jun', 'Set', 'Dic'] * 4
    ax.set_xticks(df_ventas_sp.fecha_pedido)
    ax.set_xticklabels(x_labels, ha='center')

    # Ajustando el eje x colocando un nivel encima de la granularidad por medio de la exposición del año
    texto = [16 * " " + str(i) + 16 * " " for i in range(2019, 2023)]
    ax.text(ax.get_xbound()[0] + 35, -80,
    f'|{texto[0]}|{texto[1]}|{texto[2]}|{texto[3]}|', fontsize=14, color='#05114a')

    # Etiquetas trimestrales (...)
    for i in range(3, len(df_ventas_sp["total_venta"]), 4):
        ax.text(x=ax.get_xticks()[i], y=df_ventas_sp['total_venta'][i] + 20,
        s=f"${df_ventas_sp['total_venta'][i].round(1)}mil", color='#078c62', weight='bold', fontsize=10)
        ax.text(x=ax.get_xticks()[i] + 118, y=df_ventas_sp['total_venta'][i] + 20,
        s=f"(4º tri {2019 + int(i/4)})", color='#292a2e', weight='bold', fontsize=10)

pregunta_5()


6. ¿Existe otro insight que puedas proporcionar?